In [1]:
from anytree import Node, RenderTree, NodeMixin, search, PreOrderIter
from scipy.stats import chi2_contingency
import pandas as pd
import numpy as np

from optbinning import MulticlassOptimalBinning #http://gnpalencia.org/optbinning/binning_multiclass.html
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter
import rpy2.rinterface as rinterface
from rpy2.robjects.packages import importr, data
from itertools import combinations

from rpy2.robjects import pandas2ri

from cart import *


In [2]:
'''
train = pd.read_csv("Carseats_train.csv")
test = pd.read_csv("Carseats_test.csv")
print(train.shape)

high_train = []

for i in range(train.shape[0]):
    if train["Sales"][i] > 8:
        high_train.append("Yes")
    else:
        high_train.append("No")

train["High"] = high_train
del train["Sales"]
del train["Unnamed: 0"]

indici = np.arange(0, len(high_train))

high_test = []
for i in range(test.shape[0]):
    if test["Sales"][i] > 8:
        high_test.append("Yes")
    else:
        high_test.append("No")

test["High"] = high_test
del test["Sales"]
del test["Unnamed: 0"]

feature_var = "High"
num_var = ["CompPrice", "Income", "Advertising", "Population", "Price", "Age", "Education"]
cat_var = ["ShelveLoc", "Urban", "US"]

display(train.head())


y = train["High"].copy()
del train["High"]
features = train[num_var]
features_names = num_var
n_features = train[cat_var]
n_features_names = cat_var
'''

'\ntrain = pd.read_csv("Carseats_train.csv")\ntest = pd.read_csv("Carseats_test.csv")\nprint(train.shape)\n\nhigh_train = []\n\nfor i in range(train.shape[0]):\n    if train["Sales"][i] > 8:\n        high_train.append("Yes")\n    else:\n        high_train.append("No")\n\ntrain["High"] = high_train\ndel train["Sales"]\ndel train["Unnamed: 0"]\n\nindici = np.arange(0, len(high_train))\n\nhigh_test = []\nfor i in range(test.shape[0]):\n    if test["Sales"][i] > 8:\n        high_test.append("Yes")\n    else:\n        high_test.append("No")\n\ntest["High"] = high_test\ndel test["Sales"]\ndel test["Unnamed: 0"]\n\nfeature_var = "High"\nnum_var = ["CompPrice", "Income", "Advertising", "Population", "Price", "Age", "Education"]\ncat_var = ["ShelveLoc", "Urban", "US"]\n\ndisplay(train.head())\n\n\ny = train["High"].copy()\ndel train["High"]\nfeatures = train[num_var]\nfeatures_names = num_var\nn_features = train[cat_var]\nn_features_names = cat_var\n'

In [3]:
'''
my_tree = MyNodeClass('n1', indici) 
cart = CART(y,features,features_names,n_features,n_features_names,impurity_fn = "gini",problem="classifier",method = "CART",max_level = 10, min_cases_parent= 10,min_cases_child= 5) 
cart.growing_tree(my_tree)
cart.print_tree() 
'''

'\nmy_tree = MyNodeClass(\'n1\', indici) \ncart = CART(y,features,features_names,n_features,n_features_names,impurity_fn = "gini",problem="classifier",method = "CART",max_level = 10, min_cases_parent= 10,min_cases_child= 5) \ncart.growing_tree(my_tree)\ncart.print_tree() \n'

In [4]:
#may need to bin numerical vars
#i think only works for categorical vars or binned vars 

In [5]:
pd.options.display.max_columns = None
dataframe = pd.read_table('lpmc.dat')
print(dataframe.shape)
dataframe.head()

(81086, 32)


,trip_id,household_id,person_n,trip_n,travel_mode,purpose,fueltype,faretype,bus_scale,survey_year,travel_year,travel_month,travel_date,day_of_week,start_time,age,female,driving_license,car_ownership,distance,dur_walking,dur_cycling,dur_pt_access,dur_pt_rail,dur_pt_bus,dur_pt_int,pt_interchanges,dur_driving,cost_transit,cost_driving_fuel,cost_driving_ccharge,driving_traffic_percent
0,0,0,0,0,4,3,1,1,1.0,1,2012,4,1,7,10.000000,56,1,1,2,778,0.218056,0.092222,0.134444,0.0,0.016667,0.0,0,0.052222,1.5,0.14,0.0,0.111702
1,1,0,0,1,4,3,1,1,1.0,1,2012,4,1,7,13.000000,56,1,1,2,778,0.188889,0.061944,0.109444,0.0,0.055556,0.0,0,0.059444,1.5,0.15,0.0,0.112150
2,2,0,0,2,4,3,1,1,1.0,1,2012,4,1,7,17.000000,56,1,1,2,4562,1.037778,0.355833,0.203056,0.0,0.210278,0.0,0,0.236667,1.5,0.79,0.0,0.203052
3,3,0,0,3,4,3,1,1,1.0,1,2012,4,1,7,17.350000,56,1,1,2,4562,1.039444,0.384444,0.205556,0.0,0.258611,0.0,0,0.233333,1.5,0.78,0.0,0.160714
4,4,0,1,2,4,3,1,4,1.0,1,2012,4,1,7,19.583333,54,0,1,2,4562,1.037778,0.355833,0.203056,0.0,0.189444,0.0,0,0.229167,1.5,0.78,0.0,0.130909


In [6]:
#binning age

groupDict={'0-18':0,'19-30':0,'31-65':0, '66+': 0}

for i in dataframe['age']:
    if i>=0 and i<=10:
        groupDict['0-18']+=1
    elif i>=19 and i<=30:
        groupDict['19-30']+=1
    elif i>=31 and i<=65:
        groupDict['31-65']+=1
    else:
        groupDict['66+']+=1

dataframe.loc[dataframe['age']<=18, 'age_group'] = '0-18'
dataframe.loc[dataframe['age'].between(19,30), 'age_group'] = '19-30'
dataframe.loc[dataframe['age'].between(31,65), 'age_group'] = '31-65'
dataframe.loc[dataframe['age']>65, 'age_group'] = '66+'
dataframe.drop(['age'], axis= 1, inplace = True)


#binning distance

dataframe.loc[dataframe['distance']< 658.50, 'distance_group'] = 'distance01'
dataframe.loc[dataframe['distance'].between(658.50,861.49), 'distance_group'] = 'distance02'
dataframe.loc[dataframe['distance'].between(861.50,1214.49), 'distance_group'] = 'distance03'
dataframe.loc[dataframe['distance'].between(1214.50,1514.49 ), 'distance_group'] = 'distance04'
dataframe.loc[dataframe['distance'].between(1514.50, 2156.49), 'distance_group'] = 'distance05'
dataframe.loc[dataframe['distance'].between(2156.50, 2759.49), 'distance_group'] = 'distance06'
dataframe.loc[dataframe['distance'].between(2759.50, 4391.49), 'distance_group'] = 'distance07'
dataframe.loc[dataframe['distance'].between(4391.50, 5369.49), 'distance_group'] = 'distance08'
dataframe.loc[dataframe['distance'].between(5369.50, 10087.49), 'distance_group'] = 'distance09'
dataframe.loc[dataframe['distance'].between(10087.50, 14695.49), 'distance_group'] = 'distance10'
dataframe.loc[dataframe['distance']>=14695.50, 'distance_group'] = 'distance11'
dataframe.drop('distance', axis = 1, inplace = True)

#binning start time

dataframe.loc[dataframe['start_time']<8.091666, 'starttime_group'] = 'early_morning'
dataframe.loc[dataframe['start_time'].between(8.091666,10.00833320), 'starttime_group'] = 'morning'
dataframe.loc[dataframe['start_time'].between(10.008333211,18.50833320), 'starttime_group'] = 'afternoon'
dataframe.loc[dataframe['start_time']> 18.50833321, 'starttime_group'] = 'evening'
dataframe.drop('start_time', axis = 1, inplace = True)

#binning cost transit

dataframe.loc[dataframe['cost_transit']< 1.52499998, 'cost_group'] = 'low_cost'
dataframe.loc[dataframe['cost_transit'].between(1.52499998,3.01999997), 'cost_group'] = 'medium_cost'
dataframe.loc[dataframe['cost_transit']> 3.01999998, 'cost_group'] = 'high_cost'
dataframe.drop('cost_transit', axis = 1, inplace = True)

#binning cost_driving_Fuel

dataframe.loc[dataframe['cost_driving_fuel']< 0.165, 'costdrivingfuel_group'] = 'costdrivingfuelgroup1'
dataframe.loc[dataframe['cost_driving_fuel'].between(0.165,0.204), 'costdrivingfuel_group'] = 'costdrivingfuelgroup2'
dataframe.loc[dataframe['cost_driving_fuel'].between(0.205,0.244), 'costdrivingfuel_group'] = 'costdrivingfuelgroup3'
dataframe.loc[dataframe['cost_driving_fuel'].between(0.245, 0.294), 'costdrivingfuel_group'] = 'costdrivingfuelgroup4'
dataframe.loc[dataframe['cost_driving_fuel'].between(0.295, 0.354), 'costdrivingfuel_group'] = 'costdrivingfuelgroup5'
dataframe.loc[dataframe['cost_driving_fuel'].between(0.355, 0.44499998), 'costdrivingfuel_group'] = 'costdrivingfuelgroup6'
dataframe.loc[dataframe['cost_driving_fuel'].between(0.44499999, 0.83499997), 'costdrivingfuel_group'] = 'costdrivingfuelgroup7'
dataframe.loc[dataframe['cost_driving_fuel'].between(0.83499998, 1.17499994), 'costdrivingfuel_group'] = 'costdrivingfuelgroup8'
dataframe.loc[dataframe['cost_driving_fuel'].between(1.17499995, 2.31499993), 'costdrivingfuel_group'] = 'costdrivingfuelgroup9'
dataframe.loc[dataframe['cost_driving_fuel']> 2.31499994, 'costdrivingfuel_group'] = 'costdrivingfuelgroup10'
dataframe.drop('cost_driving_fuel', axis = 1, inplace = True)

#binning driving traddic percent

dataframe.loc[dataframe['driving_traffic_percent']< 0.24422751, 'trafficperc_group'] = '0low_traffic'
dataframe.loc[dataframe['driving_traffic_percent'].between(0.24422751,0.36449698), 'trafficperc_group'] = '1mediumlow_traffic'
dataframe.loc[dataframe['driving_traffic_percent'].between(0.36449699,0.49620750), 'trafficperc_group'] = '2medium_traffic'
dataframe.loc[dataframe['driving_traffic_percent'].between(0.49620751, 0.57161700), 'trafficperc_group'] = '3mediumhigh_traffic'
dataframe.loc[dataframe['driving_traffic_percent']> 0.57161701, 'trafficperc_group'] = '4high_traffic'
dataframe.drop('driving_traffic_percent', axis = 1, inplace = True)

#binning cost_driving_ccharge

dataframe.loc[dataframe['cost_driving_ccharge']< 5.25, 'drivingcharge_group'] = 'low_charge'
dataframe.loc[dataframe['cost_driving_ccharge']> 5.25, 'drivingcharge_group'] = 'high_charge'
dataframe.drop('cost_driving_ccharge', axis = 1, inplace = True)

In [7]:
y = dataframe[['travel_mode', 'travel_year']]
X = dataframe.drop(['travel_mode', 'survey_year'], axis = 1)

print(set(X["travel_year"]))

a = X.loc[X["travel_year"]==2015]
#creating test set 

X_test= X.query('travel_year == 2015') #creates a subset, like this: X.loc[X["travel_year"]==2015]
y_test = y.query('travel_year == 2015')

X_test = X_test.drop(['travel_year'], axis =1).copy()
y_test = y_test.drop(['travel_year'], axis =1).copy()

#create training set 

X_train_1 = X.query('travel_year == 2012')
y_train_1 = y.query('travel_year == 2012')
X_train_2 = X.query('travel_year == 2013')
y_train_2 = y.query('travel_year == 2013')
X_train_3 = X.query('travel_year == 2014')
y_train_3 = y.query('travel_year == 2014')

X_train = pd.concat([X_train_1, X_train_2, X_train_3])
y_train = pd.concat([y_train_1, y_train_2, y_train_3])

X_train = X_train.drop(['travel_year'], axis = 1)
y_train = y_train.drop(['travel_year'], axis = 1)


data_to_shuffle = pd.concat([X_train, y_train], axis = 1)
data = data_to_shuffle.sample(frac=1).reset_index(drop=True) #sampling returning all, and resetting index , no seed?
y_train = data['travel_mode']
X_train = data.drop(['travel_mode'], axis = 1)

#feature selection

to_drop = ['trip_id', 'household_id', 'person_n', 'trip_n', 'travel_date', 'dur_pt_int',
          'dur_pt_rail', 'dur_pt_access', 'dur_pt_bus', 'dur_walking', 'dur_cycling', 'dur_driving']
print(f'Number of columns dropped: {len(to_drop)}')
X_train.drop(to_drop, axis = 1, inplace = True)

X_test.drop(to_drop, axis = 1, inplace = True)

predictors = X_test.columns
y = y_test #is the whole vector
print(predictors)

datatest = pd.concat([y_test, X_test], axis = 1).reset_index()
datatest.drop('index', axis = 1, inplace = True)
datatest.to_csv("data_test.csv", index = False)

datatraining = pd.concat([y_train, X_train], axis = 1).reset_index()
datatraining.drop('index', axis = 1, inplace = True)
datatraining.to_csv("data_training.csv", index = False)


{2012, 2013, 2014, 2015}
Number of columns dropped: 12
Index(['purpose', 'fueltype', 'faretype', 'bus_scale', 'travel_month',
       'day_of_week', 'female', 'driving_license', 'car_ownership',
       'pt_interchanges', 'age_group', 'distance_group', 'starttime_group',
       'cost_group', 'costdrivingfuel_group', 'trafficperc_group',
       'drivingcharge_group'],
      dtype='object')


In [8]:
display(y_train)
print(X_train.shape)
X_train.head()

0        1
1        3
2        4
3        4
4        4
        ..
76098    1
76099    4
76100    4
76101    3
76102    4
Name: travel_mode, Length: 76103, dtype: int64

(76103, 17)


,purpose,fueltype,faretype,bus_scale,travel_month,day_of_week,female,driving_license,car_ownership,pt_interchanges,age_group,distance_group,starttime_group,cost_group,costdrivingfuel_group,trafficperc_group,drivingcharge_group
0,3,2,1,1.0,12,6,1,1,2,0,31-65,distance01,early_morning,low_cost,costdrivingfuelgroup1,0low_traffic,low_charge
1,3,6,5,0.0,4,5,1,0,0,1,66+,distance06,afternoon,low_cost,costdrivingfuelgroup6,2medium_traffic,low_charge
2,3,6,1,1.0,10,5,1,1,2,0,31-65,distance06,afternoon,low_cost,costdrivingfuelgroup6,2medium_traffic,low_charge
3,3,1,1,1.0,10,6,1,1,2,2,31-65,distance11,evening,high_cost,costdrivingfuelgroup10,2medium_traffic,low_charge
4,3,6,1,1.0,5,3,0,1,2,0,31-65,distance03,evening,low_cost,costdrivingfuelgroup5,0low_traffic,low_charge


In [9]:
X_train.reset_index(inplace = True, drop = True)
y_train.reset_index(inplace = True, drop = True)

X_train=X_train.applymap(str)
#y_train=y_train.astype(str)

n_features = X_train
n_features_names = list(X_train.columns)
indici = np.arange(0, len(y_train))
y = y_train





In [10]:
#need to bin num vars and have features, features_names empty

#reched tau at each new split


#not doing optbinning internally  maybe could

features = []
features_names = []


#features, features_names, n_features, n_features_names, y

negative node prop gain?

why its selecting only 1 group for splits??

max_k as a suser input

next things!

In [11]:
from cart import *

my_tree = MyNodeClass('n1', indici) 
cart = CART(y,features,features_names,n_features,n_features_names,impurity_fn = "gini",problem="classifier",method = "LATENT-BUDGET-TREE",max_level = 2, min_cases_parent= 1000,min_cases_child= 500) 
cart.growing_tree(my_tree, max_k = 4)


level 0
ordered_list [(0.15474918705089039, 'distance_group'), (0.14617035023980365, 'car_ownership'), (0.1379897440993309, 'costdrivingfuel_group'), (0.12264490922414328, 'fueltype'), (0.09215576558991657, 'trafficperc_group'), (0.049534112303858235, 'drivingcharge_group'), (0.04715656948131236, 'driving_license'), (0.03798279928412857, 'cost_group'), (0.02975188421267667, 'pt_interchanges'), (0.02915094880544687, 'purpose'), (0.01466356106872606, 'age_group'), (0.010830641084993264, 'faretype'), (0.009374727190879659, 'day_of_week'), (0.009200017796845624, 'starttime_group'), (0.005664939369277348, 'bus_scale'), (0.0013902898247796828, 'travel_month'), (0.0004468587610848126, 'female')]
[['distance04', 'distance05', 'distance06', 'distance07', 'distance08', 'distance09', 'distance10', 'distance11'], ['0'], ['costdrivingfuelgroup10', 'costdrivingfuelgroup4', 'costdrivingfuelgroup5', 'costdrivingfuelgroup6', 'costdrivingfuelgroup7', 'costdrivingfuelgroup8', 'costdrivingfuelgroup9'], ['

R[write to console]: Error in solve.default(Tb) : 'a' (1 x 2) must be square



ordered_list [(0.08367221000102512, 'distance_group'), (0.038060766159101686, 'costdrivingfuel_group'), (0.020920702240592094, 'trafficperc_group'), (0.019989210163592064, 'driving_license'), (0.016468831049876363, 'drivingcharge_group'), (0.01491752744803404, 'car_ownership'), (0.012381603988868875, 'purpose'), (0.009840454196319171, 'age_group'), (0.008980562470867872, 'faretype'), (0.007917728520891442, 'starttime_group'), (0.004403243434644251, 'bus_scale'), (0.004134420975967916, 'travel_month'), (0.004022707711027999, 'fueltype'), (0.0027118352912042874, 'cost_group'), (0.002352104384773975, 'day_of_week'), (0.0003734202552757896, 'female'), (0.00016933325160391208, 'pt_interchanges')]
Error in LBA function
[['costdrivingfuelgroup1', 'costdrivingfuelgroup10', 'costdrivingfuelgroup2', 'costdrivingfuelgroup3', 'costdrivingfuelgroup4', 'costdrivingfuelgroup5', 'costdrivingfuelgroup6', 'costdrivingfuelgroup7', 'costdrivingfuelgroup8']] [-0.29527707] ['costdrivingfuel_group'] 0
i find

((None, (None, None)), ((None, None), None))

In [12]:
X_test.reset_index(inplace = True, drop = True)
y_test.reset_index(inplace = True, drop = True)

X_test=X_test.applymap(str)
#y_test=y_test.applymap(str)

y_test = y_test["travel_mode"] #y_test must be a series (not a dataframe)


features_test = []
n_features_test = X_test


In [13]:
#to use eval, the datatype need to be str

#alpha = cart.pruning(features_test, n_features_test, y_test)
#cart.print_alpha(alpha) 

In [14]:

pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 3000)


cart.print_tree(table = True, html= False) #returns panda dataframe atm 

#all node is leaf, then fathers

#can click and drag on graph to zoom in

,Node,Node_type,Variable_split,n,Impurity_value,Class Probabilities
0,n1,Parent,car_ownership in '['0']',76103,0.65,"{1: 0.18, 2: 0.03, 3: 0.35, 4: 0.44}"
1,n2,Parent,"distance_group in '['distance04', 'distance05', 'distance06', 'distance07', 'distance08', 'distance09', 'distance10', 'distance11']'",53924,0.57,"{1: 0.14, 2: 0.02, 3: 0.24, 4: 0.59}"
2,n4,Child,"costdrivingfuel_group in '['costdrivingfuelgroup1', 'costdrivingfuelgroup10', 'costdrivingfuelgroup2', 'costdrivingfuelgroup3', 'costdrivingfuelgroup4', 'costdrivingfuelgroup5', 'costdrivingfuelgroup6', 'costdrivingfuelgroup7', 'costdrivingfuelgroup8']'",11453,0.57,"{1: 0.48, 2: 0.03, 3: 0.05, 4: 0.43}"
3,n5,Parent,"trafficperc_group in '['0low_traffic', '1mediumlow_traffic', '2medium_traffic']'",42471,0.51,"{1: 0.05, 2: 0.02, 3: 0.3, 4: 0.63}"
4,n10,Child,None,9612,0.52,"{1: 0.03, 2: 0.04, 3: 0.62, 4: 0.31}"
5,n11,Child,None,32859,0.43,"{1: 0.05, 2: 0.02, 3: 0.2, 4: 0.73}"
6,n3,Parent,"distance_group in '['distance04', 'distance05', 'distance06', 'distance07', 'distance08', 'distance09', 'distance10', 'distance11']'",22179,0.54,"{1: 0.26, 2: 0.04, 3: 0.62, 4: 0.08}"
7,n7,Child,"distance_group in '['distance04', 'distance05', 'distance06', 'distance07', 'distance08', 'distance09', 'distance10', 'distance11']'",16307,0.39,"{1: 0.1, 2: 0.05, 3: 0.77, 4: 0.09}"
8,n6,Parent,distance_group in '['distance01']',5872,0.43,"{1: 0.73, 2: 0.03, 3: 0.2, 4: 0.05}"
9,n12,Child,None,3656,0.52,"{1: 0.63, 2: 0.03, 3: 0.28, 4: 0.06}"


In [15]:
set(n_features["car_ownership"])

{'0', '1', '2'}

In [16]:
'''
my_tree = MyNodeClass('n1', indici) 
cart = CART(y,features,features_names,n_features,n_features_names,impurity_fn = "tau",problem="classifier",method = "FAST",max_level = 2, min_cases_parent= 1000,min_cases_child= 500) 
cart.growing_tree(my_tree)
'''

level 0
i find new_split :  [['car_ownership', ('2', '1'), 0.13399232440569822, 0]] start
node_proportion_total  2.3077407093437725e-05
level 1
i find new_split :  [['distance_group', ('distance01', 'distance02', 'distance03', 'distance04'), 0.2596484530717114, 1]] left
node_proportion_total  3.0296196875331443e-05
Node_proportion_gain  -0.902696542571712
level 2
i find new_split :  [['distance_group', ('distance05', 'distance06'), 0.034313336487000204, 2]] left
node_proportion_total  3.959979420028552e-05
Node_proportion_gain  -1.1633979368370417
level 3
level 3
level 2
i find new_split :  [['distance_group', ('distance01', 'distance02'), 0.09618524728113152, 2]] right
node_proportion_total  4.803579605018784e-05
Node_proportion_gain  -1.054906699476944
level 3
level 3
level 1
i find new_split :  [['distance_group', ('distance01', 'distance02', 'distance03'), 0.08504092945236022, 1]] right
node_proportion_total  5.759496349068217e-05
Node_proportion_gain  -1.1953565153042365
level 2
i

(((None, None), (None, None)), ((None, None), (None, None)))

In [17]:
cart.print_tree(table = False, html= False) #returns panda dataframe atm 


In [18]:
# Error in solve.default(Tb) : 'a' (4 x 2) must be square

In [19]:
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr, data

#sink("/dev/null")    # now suppresses
#sink()  

robjects.r("library(utils, quietly = TRUE)")
robjects.r("library(base, quietly = TRUE)")
robjects.r("suppressWarnings(install.packages('lba', quiet = TRUE))")
robjects.r("suppressMessages(library(lba, quietly = TRUE))")


#capture.output(cat("Hi\n"), file = nullfile())


'''
utils = importr('utils')  
utils.chooseCRANmirror(ind=1) 
utils.install_packages("lba")
lba = importr("lba")
base = importr('base')
'''



'\nutils = importr(\'utils\')  \nutils.chooseCRANmirror(ind=1) \nutils.install_packages("lba")\nlba = importr("lba")\nbase = importr(\'base\')\n'

In [20]:

cont = pd.crosstab(index = n_features["distance_group"], columns= y, normalize = 'index')
#converts into an r dataframe
with (robjects.default_converter + pandas2ri.converter).context():
    cont_r = robjects.conversion.get_conversion().py2rpy(cont)

In [21]:
robjects.r.assign("cont_r", cont_r)
robjects.r("cont_r <-  as.matrix(cont_r)")

robjects.r("out <- lba(cont_r, K = 2 , what = 'outer', method = 'ls', trace.lba = FALSE)")
print(robjects.r("out$val_func"))

robjects.r("error <- out$val_func")
robjects.r("alpha <- out$A")
robjects.r("beta <- out$B")

#out = lba.lba(base.as_matrix(cont_r), K = 2 , what = 'outer', method = 'ls')

[1] 0.04655422



In [22]:
alpha  = robjects.r('alpha')
alpha=np.asarray(alpha)
alpha

array([[1.00000000e-05, 1.04221785e+00],
       [2.39506090e-01, 7.60493910e-01],
       [4.30284935e-01, 5.69715065e-01],
       [6.19840044e-01, 3.80159956e-01],
       [7.76579776e-01, 2.23420224e-01],
       [8.64604575e-01, 1.35395425e-01],
       [9.33930790e-01, 6.60692096e-02],
       [9.68200365e-01, 3.17996347e-02],
       [9.98007599e-01, 1.99240135e-03],
       [1.03366841e+00, 1.00000000e-05],
       [1.04687187e+00, 1.00000000e-05]])

In [23]:
robjects.r("library(utils)")
robjects.r("library(base)")
robjects.r("library(lba)")

